In [3]:
DATABASE_HOST="172.22.0.2"
DATABASE_DATABASE="fabricexplorer"
DATABASE_USERNAME="hppoc"
DATABASE_PASSWORD="password"

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import bamboolib as bam
import matplotlib.pyplot as ptlib
import plotly.express as px
import json

# list all user defined tables and schemas
# "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'information_schema' AND schemaname != 'pg_catalog';"

#initializing resources
engine = create_engine('postgresql://'+DATABASE_USERNAME+':'+DATABASE_PASSWORD+'@'+DATABASE_HOST+':5432/'+DATABASE_DATABASE)

txQuery = "SELECT * FROM transactions"
txDf = pd.read_sql(txQuery,con=engine)
# txDf.to_csv('backup.csv')
txDf=txDf[txDf['read_set'].isna()==False]
txDf=txDf[txDf['chaincodename'].str.contains('_lifecycle')==False]
txDf=txDf[txDf['type'].str.contains('CONFIG')==False]
txDf=txDf.drop(columns=["txhash","chaincode_id","endorser_msp_id","type","createdt","channel_genesis_hash","envelope_signature","creator_id_bytes", "creator_nonce", "payload_extension","tx_response","payload_proposal_hash","endorser_id_bytes","endorser_signature", "network_name"])
txDf

      id  blockid chaincodename  status     creator_msp_id  \
0      6        3          lscc   200.0  CentralBankOrgMSP   
4     10        4          cbdc   200.0  CentralBankOrgMSP   
5     11        5          cbdc   200.0           FIOrgMSP   
6     12        6          cbdc   200.0           FIOrgMSP   
7     13        7          cbdc   200.0           FIOrgMSP   
..   ...      ...           ...     ...                ...   
578  584      225          cbdc   200.0           FIOrgMSP   
579  585      225          cbdc   200.0           FIOrgMSP   
580  586      226          cbdc   200.0           FIOrgMSP   
581  587      227          cbdc   200.0           FIOrgMSP   
582  588      228          cbdc   200.0           FIOrgMSP   

                                              read_set  \
0    [{'chaincode': 'lscc', 'set': [{'key': 'cbdc'}]}]   
4    [{'chaincode': 'cbdc', 'set': []}, {'chaincode...   
5    [{'chaincode': 'cbdc', 'set': [{'key': 'A0x73b...   
6    [{'chaincode': 'cb

In [4]:
# txDf['read_set']=read_set
# txDf['write_set']=write_set
txDf = txDf.explode('read_set')
txDf = txDf.explode('write_set')
txDf = txDf.loc[(txDf['read_set'].astype('string').str.contains('version', case=False, regex=False, na=False))]
txDf

      id  blockid chaincodename  status     creator_msp_id  \
4     10        4          cbdc   200.0  CentralBankOrgMSP   
4     10        4          cbdc   200.0  CentralBankOrgMSP   
5     11        5          cbdc   200.0           FIOrgMSP   
5     11        5          cbdc   200.0           FIOrgMSP   
6     12        6          cbdc   200.0           FIOrgMSP   
..   ...      ...           ...     ...                ...   
581  587      227          cbdc   200.0           FIOrgMSP   
582  588      228          cbdc   200.0           FIOrgMSP   
582  588      228          cbdc   200.0           FIOrgMSP   
582  588      228          cbdc   200.0           FIOrgMSP   
582  588      228          cbdc   200.0           FIOrgMSP   

                                              read_set  \
4    {'chaincode': 'lscc', 'set': [{'key': 'cbdc', ...   
4    {'chaincode': 'lscc', 'set': [{'key': 'cbdc', ...   
5    {'chaincode': 'lscc', 'set': [{'key': 'cbdc', ...   
5    {'chaincode': 'lsc

In [5]:
txDf['r_chaincode'] =txDf['read_set'].apply(lambda x: x['chaincode'])
txDf['rs'] =txDf['read_set'].apply(lambda x: x['set'])
txDf = txDf.drop(columns=['chaincodename','read_set'])
txDf['w_chaincode'] =txDf['write_set'].apply(lambda x: x['chaincode'])
txDf['ws'] =txDf['write_set'].apply(lambda x: x['set'])
txDf = txDf.drop(columns=['write_set'])
txDf

      id  blockid  status     creator_msp_id validation_code  \
4     10        4   200.0  CentralBankOrgMSP           VALID   
4     10        4   200.0  CentralBankOrgMSP           VALID   
5     11        5   200.0           FIOrgMSP           VALID   
5     11        5   200.0           FIOrgMSP           VALID   
6     12        6   200.0           FIOrgMSP           VALID   
..   ...      ...     ...                ...             ...   
581  587      227   200.0           FIOrgMSP           VALID   
582  588      228   200.0           FIOrgMSP           VALID   
582  588      228   200.0           FIOrgMSP           VALID   
582  588      228   200.0           FIOrgMSP           VALID   
582  588      228   200.0           FIOrgMSP           VALID   

                              chaincode_proposal_input r_chaincode  \
4    7365740690746906741000776106365,46490726705350...        lscc   
4    7365740690746906741000776106365,46490726705350...        lscc   
5    6372656174654164

In [6]:
def classifyWrite(ws):
    if str(ws)=='nan':
        return np.nan
    elif ws['is_delete']:
        return 'DELETE'
    else:
        return 'WRITE'
txDfr = txDf.drop(columns=['w_chaincode','ws']).explode('rs')
txDfr = txDfr.rename(columns={'r_chaincode': 'chaincode','rs':'key'})
txDfr['access_type'] = txDfr['key'].apply(lambda x: 'READ')
txDfr['version_block'] = txDfr['key'].apply(lambda x: np.nan if str(x) == 'nan' else x['version']['block_num']['low'])
txDfr['version_tx'] = txDfr['key'].apply(lambda x: np.nan if str(x) == 'nan' else x['version']['tx_num']['low'])
txDfw = txDf.drop(columns=['r_chaincode','rs']).explode('ws')
txDfw = txDfw.rename(columns={'w_chaincode': 'chaincode','ws':'key'})
txDfw['access_type'] = txDfw['key'].apply(lambda x: classifyWrite(x))
txDf = txDfr.append(txDfw).reset_index(drop=True).rename(columns={'id':'txid'})
txDf['key']=txDf['key'].apply(lambda x:np.nan if str(x) == 'nan' else x['key'])
# Step: Drop missing values in ['key']
txDf = txDf.dropna(subset=['key'])
# Step: Drop rows where chaincode is one of: lscc
txDf = txDf.loc[~(txDf['chaincode'].isin(['lscc','_lifecycle']))]
txDf

      txid  blockid  status creator_msp_id validation_code  \
396    208      105   200.0       FIOrgMSP           VALID   
397    208      105   200.0       FIOrgMSP           VALID   
398    208      105   200.0       FIOrgMSP           VALID   
399    208      105   200.0       FIOrgMSP           VALID   
402    209      106   200.0       FIOrgMSP           VALID   
...    ...      ...     ...            ...             ...   
5524   587      227   200.0       FIOrgMSP           VALID   
5526   588      228   200.0       FIOrgMSP           VALID   
5527   588      228   200.0       FIOrgMSP           VALID   
5529   588      228   200.0       FIOrgMSP           VALID   
5530   588      228   200.0       FIOrgMSP           VALID   

                               chaincode_proposal_input chaincode  \
396   069074550697473,307837336261343136656263623030...      cbdc   
397   069074550697473,307837336261343136656263623030...      cbdc   
398   069074550697473,30783733626134313665626362

In [31]:
txDfr=txDf[txDf['access_type']=='READ']
labels={"x":"Read keys","y":"Number of reads"}
reads= txDfr.groupby(['key']).size()
reads = pd.DataFrame(reads)
reads.columns = [str(column) for column in reads.columns]
reads = reads.reset_index()
reads = reads.sort_values(by=['0'], ascending=[False])
fig = px.bar(reads, x='key', y='0', labels={'0':'reads'})
fig

In [8]:
txDfw=txDf[txDf['access_type']=='WRITE']
labels={"x":"Written keys","y":"Number of writes"}
fig = px.bar(x=txDfw['key'].unique(),y=txDfw['key'].value_counts(),labels=labels)
fig

In [9]:
labels={"x":"Validation code","y":"Qty"}
codes=txDfr.groupby(['validation_code']).size()
df = pd.DataFrame(codes)
df = df.reset_index()
df.columns = [str(column) for column in df.columns]
df = df.sort_values(by=['0'], ascending=[False])
labels={"x":"Key","y":"Ocurrence in transactions with MVCC"}
fig = px.bar(x=df['validation_code'], y=df['0'],labels=labels)
fig

In [10]:
df = pd.DataFrame(txDfr.groupby(['key','validation_code']).size())
df = df.reset_index()
df.columns = [str(column) for column in df.columns]
# import plotly.express as px
fig = px.bar(df, x='key', y='0', color='validation_code', barmode='group',labels={'0':"Quantity"})
fig

In [11]:
ratio = df.loc[~(df['validation_code'].isin(['ENDORSEMENT_POLICY_FAILURE']))]
ratio = ratio.pivot_table(index=['key'], 
            columns=['validation_code'], values='0').fillna(0)
ratio['ratio']=ratio['MVCC_READ_CONFLICT']/ratio['VALID']
ratio = ratio.reset_index()
# Step: Sort column(s) ratio descending (Z-A)
ratio = ratio.sort_values(by=['ratio'], ascending=[False])
ratio
import plotly.express as px
fig = px.bar(ratio, x='key', y='ratio',title='Ratio of MVCC/Valid')
fig

In [12]:
mvcc=txDfr[txDfr['validation_code']=='MVCC_READ_CONFLICT'].groupby(['key']).size()
df = pd.DataFrame(mvcc)
df = df.reset_index()
df.columns = [str(column) for column in df.columns]
df = df.sort_values(by=['0'], ascending=[False])
labels={"x":"Key","y":"Ocurrence in transactions with MVCC"}
fig = px.bar(x=df['key'], y=df['0'], labels=labels)
fig

In [13]:
txDfr

      txid  blockid  status creator_msp_id validation_code  \
396    208      105   200.0       FIOrgMSP           VALID   
397    208      105   200.0       FIOrgMSP           VALID   
398    208      105   200.0       FIOrgMSP           VALID   
399    208      105   200.0       FIOrgMSP           VALID   
402    209      106   200.0       FIOrgMSP           VALID   
...    ...      ...     ...            ...             ...   
2657   587      227   200.0       FIOrgMSP           VALID   
2660   588      228   200.0       FIOrgMSP           VALID   
2661   588      228   200.0       FIOrgMSP           VALID   
2662   588      228   200.0       FIOrgMSP           VALID   
2663   588      228   200.0       FIOrgMSP           VALID   

                               chaincode_proposal_input chaincode  \
396   069074550697473,307837336261343136656263623030...      cbdc   
397   069074550697473,307837336261343136656263623030...      cbdc   
398   069074550697473,30783733626134313665626362

In [14]:
labels={"x":"Chaincode","y":"Key reads"}
fig = px.bar(x=txDfr['chaincode'].unique(),y=txDfr['chaincode'].value_counts(),labels=labels)
fig

In [15]:
labels={"x":"Chaincode","y":"Key writes"}
fig = px.bar(x=txDfw['chaincode'].unique(),y=txDfw['chaincode'].value_counts(),labels=labels)
fig

In [16]:
query = "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'information_schema' AND schemaname != 'pg_catalog';"
# query = "select * from peer"
idkdf = pd.read_sql(query,con=engine)
idkdf

  schemaname           tablename tableowner tablespace  hasindexes  hasrules  \
0     public              blocks      hppoc       None        True     False   
1     public          chaincodes      hppoc       None        True     False   
2     public             channel      hppoc       None        True     False   
3     public             orderer      hppoc       None        True     False   
4     public                peer      hppoc       None        True     False   
5     public  peer_ref_chaincode      hppoc       None        True     False   
6     public    peer_ref_channel      hppoc       None        True     False   
7     public        transactions      hppoc       None        True     False   
8     public               users      hppoc       None        True     False   
9     public          write_lock      hppoc       None        True     False   

   hastriggers  rowsecurity  
0        False        False  
1        False        False  
2        False        False  

In [17]:
import binascii
import curses
# binascii.hexlify('a'.encode('utf8'))
# "e".encode('utf8').hex()
# bytes.fromhex('69').decode()
# bytes.fromhex('690760065').decode('utf8')
# binascii.unhexlify('6900760065').decode('utf8')
print(str(b"690760065",'utf8'))

690760065
